In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import torch
import os

In [ ]:
GRAPHIC_DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

In [ ]:
GRAPHIC_DEVICE

In [ ]:
torch.cuda.is_available()

In [ ]:
heights = [189, 170, 189, 163, 183, 171, 185,
168, 173, 183, 173, 173, 175, 178,
183, 193, 178, 173, 174, 183, 183,
180, 168, 180, 170, 178, 182, 180,
183, 178, 182, 188, 175, 179, 183,
193, 182, 183, 177, 185, 188, 188,
182, 185, 191, 183]

In [ ]:
heights_tensor = torch.tensor(heights, dtype=torch.int32)

In [ ]:
tensor1 = torch.zeros(2, 3)

In [ ]:
tensor1

In [ ]:
tensor2 = torch.ones(1, 4, 5)

In [ ]:
nparr = np.arange(10, dtype=np.int32)

In [ ]:
nparr.dtype

In [ ]:
pt_tensor = torch.from_numpy(nparr)

In [ ]:
pt_tensor.dtype

In [ ]:
heights_in_feet = heights_tensor / 30.48

In [ ]:
heights_tensor.dtype

In [ ]:
heights_2_measures = torch.cat([heights_tensor, heights_in_feet])

In [ ]:
heights_2_measures.dtype

In [ ]:
heights_reshaped = heights_2_measures.reshape(2, 46)

In [ ]:
heights_reshaped

In [ ]:
heights_reshaped.mean(dim=1)

In [ ]:
values, indices = heights_reshaped.max(dim=1)

In [ ]:
values

In [ ]:
indices

In [ ]:
import torch.nn as nn
import torchvision
import torchvision.transforms as T

In [ ]:
torch.manual_seed(2)
transform = T.Compose(
    [
        T.ToTensor(),
        T.Normalize([.5], [.5])
    ]
)
train_set = torchvision.datasets.FashionMNIST(
    root='.',
    train=True,
    download=True,
    transform=transform
)

In [ ]:
test_set = torchvision.datasets.FashionMNIST(
    root='.',
    train=False,
    download=True,
    transform=transform
)
text_labels=['t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']

In [ ]:
plt.figure(dpi=300, figsize=(8,4))
for i in range(24):
    ax = plt.subplot(3, 8, i+1)
    img = train_set[i][0]
    img = img/2 + .5
    img = img.reshape(28, 28)
    plt.imshow(img, cmap='binary')
    plt.axis('off')
    plt.title(text_labels[train_set[i][1]], fontsize=8)


In [ ]:
binary_train_set = [x for x in train_set if x[1] in (0, 9)]
binary_test_set = [x for x in test_set if x[1] in (0, 9)]

In [ ]:
import torch.utils
import torch.utils.data
torch.manual_seed(2)
batch_size = 64
binary_train_loader = torch.utils.data.DataLoader(binary_train_set, batch_size=batch_size, shuffle=True)
binary_test_loader = torch.utils.data.DataLoader(binary_test_set, batch_size=batch_size, shuffle=True)

In [ ]:
binary_model = nn.Sequential(
    nn.Linear(28 ** 2, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Dropout(p=.25),
    nn.Sigmoid()
).to(GRAPHIC_DEVICE)

In [ ]:
lr = .001
optimizer = torch.optim.Adam(binary_model.parameters(), lr=lr)
loss_fn = nn.BCELoss()

In [ ]:
for i in range(50):
    tloss = 0
    for n, (imgs, labels) in enumerate(binary_train_loader):
        imgs = imgs.reshape(-1, 28**2).to(GRAPHIC_DEVICE)
        labels = torch.FloatTensor([x if x==0 else 1 for x in labels]).reshape(-1, 1).to(GRAPHIC_DEVICE)
        preds = binary_model(imgs)
        loss = loss_fn(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        tloss += loss.detach()
    tloss /= n
    print(f"at epoch {i}, loss is {tloss}")